<a href="https://colab.research.google.com/github/rohithv-21/Hotel-booking-analysis/blob/main/Individual_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Have you ever wondered when the best time of year to book a hotel room is? Or the optimal length of stay in order to get the best daily rate? What if you wanted to predict whether or not a hotel was likely to receive a disproportionately high number of special requests? This hotel booking dataset can help you explore those questions!**

**This data set contains booking information for a city hotel and a resort hotel, and includes information such as when the booking was made, length of stay, the number of adults, children, and/or babies, and the number of available parking spaces, among other things. All personally identifying information has been removed from the data.**

**Explore and analyze the data to discover important factors that govern the bookings.**

In [1]:
# lets import the libraries required for data analysing

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# lets import the data by mounting the drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# read the csv file

file_path ="/content/drive/MyDrive/Colab Notebooks/Capstone Project 1/Rohith.V - Hotel booking analysis EDA/Hotel Bookings.csv"
hotel_booking_analysis_data = pd.read_csv(file_path)

In [6]:
# to get the top 5 rows and bottom 5 from the imported data

hotel_booking_analysis_data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [ ]:
hotel_booking_analysis_data.tail()

In [ ]:
# lets get more info about numerical data
hotel_booking_analysis_data.describe()

In [ ]:
# find count of number of rows and columns
hotel_booking_analysis_data.shape

From the data we have 119390 rows and 32 columns

In [ ]:
# lets analyze the data basically

hotel_booking_analysis_data.info()

Create a copy of the given dataset, so that our original dataset remains unchanged

In [ ]:
# creating the copy of the given data

df = hotel_booking_analysis_data.copy()

In [ ]:
# to find the sum of null rows 

df.isnull().sum()

# Data cleaning and handling of missing values

Lets find the percentage of null values present in the dataset

In [ ]:
# Create a function to find percentage of null values present

def percent_missing(df):
    percent_nan = df.isnull().sum() / len(df) * 100
    percent_nan = percent_nan[percent_nan>0].sort_values(ascending = False)
    return percent_nan 

In [ ]:
percent_nan = percent_missing(df)
percent_nan

In [ ]:
x = percent_nan.plot(figsize = (10,5) , kind = "bar")
plt.title('PERCENTAGE OF NULL VALUES' , fontsize = 20)
plt.ylabel('Percentage' , fontsize = 15)
plt.xlabel('Null value Columns' , fontsize = 15)

**Inference :** Clearly from the graph company column has got 94% and agent column has got 13% of null values where country and children columns has got less than 1%.

Since the company column has got 94% of the null values it is better to drop out this column

In [ ]:
df.drop(["company"] ,axis = 1, inplace = True)

In [ ]:
df.shape

We will fill the agent id with "Agent" and null agent id with "No Agent"

In [ ]:
# First we will fill the agent ids with 'Agent'

df.loc[df.agent.isnull()==False,'agent']='Agent'

In [ ]:
# Now we will fill the null values present in agent ids with 'No agent'

df.agent.fillna('No agent',inplace=True)

In [ ]:
df[["agent"]].head()

We would fill the missing values in the country coulumns with 'Unknown' since we do not seem to find any references to other columns

In [ ]:
df.country.fillna('Unknown',inplace=True)

In [ ]:
df["country"]

We would fill the missing values in the children column with 0

In [ ]:
df.children.fillna(0,inplace=True)

In [ ]:
df[["children"]].astype("int64")

In [ ]:
# lets check again if we have any nulls present in our dataset

df.isnull().sum()

So now we got all the columns and rows with no null values.

Now our data is cleaned and missing values has been tackled

Now, lets check for duplicate data, if present lets remove those values

In [ ]:
# checking for duplicate data
df.duplicated().sum()

In [ ]:
# removing the duplicate data
df.drop_duplicates(inplace = True)

We are adding 2 Extra Columns for more clarification
1.   Total Stay in the Hotel
2.   Total number of people



In [ ]:
# Adding total staying days in hotels
df['total_stay'] = df['stays_in_weekend_nights']+df['stays_in_week_nights']
df[['total_stay']].head()

In [ ]:
# Adding total people num as column, i.e. total people num = num of adults + children + babies
df['total_people'] = df['adults']+df['children']+df['babies']
df[['total_people']].head().astype("int64")

# Performing EDA

**EDA based on Cancelation bookings**

1) Cancelation done on the basis of months

In [ ]:
# Lets Select and count the number of cancelled bookings based on months.
cancelled_data = df[df['is_canceled'] == 1]
cancelled = cancelled_data.groupby('arrival_date_month')
x = pd.DataFrame(cancelled.size()).rename(columns = {0:'total_cancelled_bookings'})

# Count the total number of bookings for each month
grouped_by_hotel = df.groupby('arrival_date_month')
total_booking = grouped_by_hotel.size()
y = pd.DataFrame(total_booking).rename(columns = {0: 'total_bookings'})
z = pd.concat([x,y], axis = 1)

# Calculating cancel percentage 
z['cancel_%'] = round((z['total_cancelled_bookings']/z['total_bookings'])*100,2)
z

In [ ]:
plt.figure(figsize = (10,5))
sns.barplot(x = z.index, y = z['cancel_%']).set_title("Hotel having High Cancelation Rate" , fontsize = 20)
plt.show()

**Inference :** From the analyzation, in the month of july and august cancelation is done more.

2) Hotels having high cancelation rate

In [ ]:
# Lets Select and count the number of cancelled bookings for each hotel.
cancelled_data_over_week_no = df[df['is_canceled'] == 1]
cancel = cancelled_data_over_week_no.groupby('hotel')
a = pd.DataFrame(cancel.size()).rename(columns = {0:'total_cancelled_bookings'})

# Count the total number of bookings for each type of hotel
grouped_hotel = df.groupby('hotel')
total_booking_done = grouped_hotel.size()
b = pd.DataFrame(total_booking_done).rename(columns = {0: 'total_bookings'})
c = pd.concat([a,b], axis = 1)

# Calculating cancel percentage 
c['cancel%'] = round((c['total_cancelled_bookings']/c['total_bookings'])*100,2)
c

In [ ]:
plt.figure(figsize = (10,5))
sns.barplot(x = c.index, y = c['cancel%']).set_title("Hotel having High Cancelation Rate" , fontsize = 20)
plt.show()

**Inference :** City hotels have higher cancelation rates (of 30%) compared to resort hotels

**EDA based on Meals**

In [ ]:
# categories of meal feature  we have in our data
df['meal'].unique()

**People at each hotel can choose from four categories of meals which they can book their respective stays with, let's briefly see what those are**
*   Undefined/SC – no meal package
*   BB – Bed & Breakfast
*   HB – Half board (breakfast and one other meal – usually dinner)
*   FB – Full board (breakfast, lunch and dinner)

In [ ]:
# Group them according to City and Resort Hotels
meal_category = df.groupby(['hotel'])['meal'].value_counts()
meal_category

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
meal_category.plot(kind= 'bar')

plt.ylabel('No. of bookings',fontsize = 15)
plt.xlabel('Hotel/Meal',fontsize = 15)
plt.title('Meals opted in each hotels',fontsize = 20)

**Inference :** Most the peoples prefer to opt BB(Bed & Breakfast) in both city and resort hotels.

From the above chart, let me explain how. We have each hotel with respective food categories and the number of booking that people are doing in each hotel with each meal category, now it is clear that, in Resort Hotel or city Hotel, people mostly prefer to go with the BB-category (i.e., bed and breakfast)

**EDA based on country**